In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from hyperopt import fmin, tpe, hp
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [33]:
# 1. Reading data from CSV
def read_csv(file_path):
    return pd.read_csv(file_path)

# 2. Creating features
def create_features(data):
    # No feature creation for this example
    return data

# 3. Training a classifier model
def train_classifier(data):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestClassifier()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    return model, accuracy

# 4. Hyperparameter tuning with Hyperopt
def objective(params):
    model = RandomForestClassifier(**params)
    score = cross_val_score(model, X, y, cv=5).mean()
    return -score  # Minimize negative accuracy

# 5. Evaluating the model on the test set
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [34]:
file_path = "IRIS.csv"
data = read_csv(file_path)

In [35]:
data

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [ ]:
file_path = "IRIS.csv"
data = read_csv(file_path)

# Create features
data = create_features(data)

# Split data into features and target
X = data.drop('species', axis=1)
y = data['species']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the pipeline
pipeline = Pipeline([
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), X.columns)
        ],
        remainder='passthrough'
    )),
    ('classifier', RandomForestClassifier())
])

# Train the model
pipeline.fit(X_train, y_train)

# Evaluate the model
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy on test set: {accuracy}")

#Brute force appraoch try with 2 for loops

# Hyperparameter tuning using Tree of Parzen Estimators (TPE)
space = {
    'n_estimators': hp.choice('n_estimators', range(10, 101)),
    'max_depth': hp.choice('max_depth', range(1, 21))
}

best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100)

# Update the pipeline with the best hyperparameters


# Train the model with the best hyperparameters


# Evaluate the updated model


Model accuracy on test set: 1.0
 70%|████████████████████████████████▏             | 70/100 [00:46<00:18,  1.60trial/s, best loss: -0.9666666666666668]

In [25]:
best_params

{'max_depth': np.int64(3), 'n_estimators': np.int64(0)}

In [26]:
pipeline

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [27]:
{'classifier_'+k : v for k, v in best_params.items()}

{'classifier_max_depth': np.int64(3), 'classifier_n_estimators': np.int64(0)}

In [28]:
update_pipeline = pipeline.set_params(**{'classifier__'+k : v for k, v in best_params.items()})

In [29]:
update_pipeline

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [30]:
update_pipeline.fit(X_train,y_train)

InvalidParameterError: The 'n_estimators' parameter of RandomForestClassifier must be an int in the range [1, inf). Got np.int64(0) instead.

In [31]:
y_pred_updated = update_pipeline.predict(X_test)
accuracy_updated = accuracy_score(y_test, y_pred_updated)
print(f"Updated Model accuracy on test set: {accuracy}")


C:\Users\NAIM\miniconda3\Lib\site-packages\sklearn\ensemble\_base.py:171: RuntimeWarning: divide by zero encountered in scalar floor_divide
  n_estimators_per_job = np.full(n_jobs, n_estimators // n_jobs, dtype=int)
C:\Users\NAIM\miniconda3\Lib\site-packages\sklearn\ensemble\_base.py:172: RuntimeWarning: divide by zero encountered in scalar remainder
  n_estimators_per_job[: n_estimators % n_jobs] += 1


ValueError: n_jobs == 0 in Parallel has no meaning